In [1]:
import pyphs as phs
import sympy as sy
import numpy as np
import os

import datetime
import pickle
import sympy as sy
import matplotlib.pyplot as plt
import h5py
sy.init_printing()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

PPTY_PHY_PARAMS = {"positive": True, "real" : True }                            
PPTY_STATE_VAR  = {"real" : True }

In [2]:
Ns = 4  # Number of subsystems


Ne = 3 # N of state per subsys
N_lambda = Ns-1 # constraints
Nx = Ns * Ne     # total N of states
Nx

piL_vec  = sy.symbols('Pi_L1:{}'.format(Ns+1), **PPTY_STATE_VAR)
piR_vec  = sy.symbols('Pi_R1:{}'.format(Ns+1), **PPTY_STATE_VAR)
xk_vec   = sy.symbols('x_k1:{}'.format(Ns+1), **PPTY_STATE_VAR)

mL_vec   = sy.symbols('m_L1:{}'.format(Ns+1), **PPTY_PHY_PARAMS)
mR_vec   = sy.symbols('m_R1:{}'.format(Ns+1), **PPTY_PHY_PARAMS)
k_vec    = sy.symbols('k_1:{}'.format(Ns+1), **PPTY_PHY_PARAMS)

params = mL_vec + mR_vec + k_vec

X = []
for i in range(Ns):
    X.append(piL_vec[i])
    X.append(piR_vec[i])
X += list(xk_vec)
X = sy.Matrix(X)
X

⎡Π_L1⎤
⎢    ⎥
⎢Π_R1⎥
⎢    ⎥
⎢Π_L2⎥
⎢    ⎥
⎢Π_R2⎥
⎢    ⎥
⎢Π_L3⎥
⎢    ⎥
⎢Π_R3⎥
⎢    ⎥
⎢Π_L4⎥
⎢    ⎥
⎢Π_R4⎥
⎢    ⎥
⎢xₖ₁ ⎥
⎢    ⎥
⎢xₖ₂ ⎥
⎢    ⎥
⎢xₖ₃ ⎥
⎢    ⎥
⎣xₖ₄ ⎦

In [3]:
bT = sy.zeros(N_lambda, Nx)
for i in range(N_lambda):
    bT[i, 1+2*i] = 1 
    bT[i, 2+i*2] = -1
b = bT.T

annul_b = sy.zeros(Nx-N_lambda,Nx)

annul_b[0,0] = 1
annul_b[Ns::, Ns + N_lambda::] = sy.eye(Ns+1)
for i in range(1,N_lambda+1):
    annul_b[i, 2*i-1] = 1 
    annul_b[i, 2*i] = 1

M = sy.zeros(Nx, Nx)
M[0:Nx-N_lambda,::] = annul_b
M[Nx-N_lambda::, ::] = (bT*b).inv()*bT

Z = M*X
for i in range(N_lambda):
    Z[i+1] = sy.symbols('Pi_{0}{1}'.format(i+1,i+2), **PPTY_STATE_VAR)
for i in range(N_lambda):
    Z[Nx-N_lambda+i] = sy.symbols('Delta_{}'.format(10*(i+1) + (i+2)))
Z.T

# We are in a linear configuration for now
Q = sy.zeros(Nx)
for i in range(Ns):
    Q[2*i, 2*i] = 1/mL_vec[i]
    Q[2*i+1, 2*i+1] = 1/mR_vec[i]
    Q[i + 2*Ns, i + 2*Ns] = k_vec[i]
HX = sy.Rational(1,2) * X.T * Q * X
HX[0]

Minv = M.inv()
QZ = Minv.T * Q * Minv
QZ

HZ = sy.Rational(1,2) * Z.T * QZ * Z
HZ = HZ[0].expand().collect(Z)
HZ

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

                                                                              
   2 ⎛  1        1   ⎞      2 ⎛  1        1   ⎞      2 ⎛  1        1   ⎞      
Δ₁₂ ⋅⎜────── + ──────⎟ + Δ₂₃ ⋅⎜────── + ──────⎟ + Δ₃₄ ⋅⎜────── + ──────⎟ + Π₁₂
     ⎝2⋅m_R1   2⋅m_L2⎠        ⎝2⋅m_R2   2⋅m_L3⎠        ⎝2⋅m_R3   2⋅m_L4⎠      

                                                                              
2 ⎛  1        1   ⎞       ⎛ Δ₁₂      Δ₁₂  ⎞      2 ⎛  1        1   ⎞       ⎛ Δ
 ⋅⎜────── + ──────⎟ + Π₁₂⋅⎜────── - ──────⎟ + Π₂₃ ⋅⎜────── + ──────⎟ + Π₂₃⋅⎜──
  ⎝8⋅m_R1   8⋅m_L2⎠       ⎝2⋅m_R1   2⋅m_L2⎠        ⎝8⋅m_R2   8⋅m_L3⎠       ⎝2⋅

                                                                      2       
₂₃      Δ₂₃  ⎞      2 ⎛  1        1   ⎞       ⎛ Δ₃₄      Δ₃₄  ⎞   Π_L1     Π_R
──── - ──────⎟ + Π₃₄ ⋅⎜────── + ──────⎟ + Π₃₄⋅⎜────── - ──────⎟ + ────── + ───
m_R2   2⋅m_L3⎠        ⎝8⋅m_R3   8⋅m_L4⎠       ⎝2⋅m_R3   2⋅m_L4⎠   2⋅m_L1   2⋅m

 2          2         2         2         2
4    

In [4]:
cstr_states = Z[Nx-N_lambda::]

cstr_eq = []
display("Find", cstr_states, "such as: ")
for i in range(N_lambda):
    cstr_eq.append(sy.Eq(0, HZ.diff(cstr_states[i])))
    display(cstr_eq[i])

'Find'

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

[Δ₁₂, Δ₂₃, Δ₃₄]

'such as: '

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

          ⎛  1        1   ⎞       ⎛  1        1   ⎞
0 = 2⋅Δ₁₂⋅⎜────── + ──────⎟ + Π₁₂⋅⎜────── - ──────⎟
          ⎝2⋅m_R1   2⋅m_L2⎠       ⎝2⋅m_R1   2⋅m_L2⎠

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

          ⎛  1        1   ⎞       ⎛  1        1   ⎞
0 = 2⋅Δ₂₃⋅⎜────── + ──────⎟ + Π₂₃⋅⎜────── - ──────⎟
          ⎝2⋅m_R2   2⋅m_L3⎠       ⎝2⋅m_R2   2⋅m_L3⎠

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

          ⎛  1        1   ⎞       ⎛  1        1   ⎞
0 = 2⋅Δ₃₄⋅⎜────── + ──────⎟ + Π₃₄⋅⎜────── - ──────⎟
          ⎝2⋅m_R3   2⋅m_L4⎠       ⎝2⋅m_R3   2⋅m_L4⎠

## Interconnexion matrix

In [5]:
Jxi = sy.Matrix([[0,0,1],[0,0,-1],[-1,1,0]])
Jxi

⎡0   0  1 ⎤
⎢         ⎥
⎢0   0  -1⎥
⎢         ⎥
⎣-1  1  0 ⎦

In [6]:
Jtot = sy.zeros(Nx)
for i in range(Ns):
    Jtot[3*i:3*i+Ne, 3*i:3*i+Ne] = Jxi
Jtot

⎡0   0  1   0   0  0   0   0  0   0   0  0 ⎤
⎢                                          ⎥
⎢0   0  -1  0   0  0   0   0  0   0   0  0 ⎥
⎢                                          ⎥
⎢-1  1  0   0   0  0   0   0  0   0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  1   0   0  0   0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  -1  0   0  0   0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   -1  1  0   0   0  0   0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  0   0   0  1   0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  0   0   0  -1  0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  0   -1  1  0   0   0  0 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  0   0   0  0   0   0  1 ⎥
⎢                                          ⎥
⎢0   0  0   0   0  0   0   0  0   0   0  -1⎥
⎢                                          ⎥
⎣0   0  0 

In [7]:
# permutation matrix: re-arranging Jxx to match X
P = sy.SparseMatrix(sy.zeros(Nx))

for i in range(Ns):
    # nu_L/ nu_R
    P[2*i, Ne*i]         = 1
    P[2*i+1, Ne*i+1]     = 1
    # Piy
    P[2*Ns+i, Ne*i+2] = 1
P

⎡1  0  0  0  0  0  0  0  0  0  0  0⎤
⎢                                  ⎥
⎢0  1  0  0  0  0  0  0  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  1  0  0  0  0  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  1  0  0  0  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  0  0  1  0  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  0  0  0  1  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  0  0  0  0  0  1  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  0  0  0  0  0  0  1  0⎥
⎢                                  ⎥
⎢0  0  1  0  0  0  0  0  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  0  1  0  0  0  0  0  0⎥
⎢                                  ⎥
⎢0  0  0  0  0  0  0  0  1  0  0  0⎥
⎢                                  ⎥
⎣0  0  0  0  0  0  0  0  0  0  0  1⎦

In [8]:
Jxx = M*P*Jtot*P.T*M.T
Jxx = Jxx[0:Nx-N_lambda, 0:Nx-N_lambda]
Jxx

⎡0   0   0   0   0  1   0   0   0 ⎤
⎢                                 ⎥
⎢0   0   0   0   0  -1  1   0   0 ⎥
⎢                                 ⎥
⎢0   0   0   0   0  0   -1  1   0 ⎥
⎢                                 ⎥
⎢0   0   0   0   0  0   0   -1  1 ⎥
⎢                                 ⎥
⎢0   0   0   0   0  0   0   0   -1⎥
⎢                                 ⎥
⎢-1  1   0   0   0  0   0   0   0 ⎥
⎢                                 ⎥
⎢0   -1  1   0   0  0   0   0   0 ⎥
⎢                                 ⎥
⎢0   0   -1  1   0  0   0   0   0 ⎥
⎢                                 ⎥
⎣0   0   0   -1  1  0   0   0   0 ⎦

## Building RHS

In [9]:
Z1 = sy.Matrix(Z[0:Nx-N_lambda])
Z2 = sy.Matrix(Z[Nx-N_lambda::])
display(Z1.T, Z2.T)

[Π_L1  Π₁₂  Π₂₃  Π₃₄  Π_R4  xₖ₁  xₖ₂  xₖ₃  xₖ₄]

[Δ₁₂  Δ₂₃  Δ₃₄]

In [10]:
QZ12 = QZ[(Nx-N_lambda)::, 0:(Nx-N_lambda)]
QZ11 = QZ[0:(Nx-N_lambda), 0:(Nx-N_lambda)]
RHS = QZ12 * sy.Matrix(Z1)
RHS

⎡    ⎛  1        1   ⎞⎤
⎢Π₁₂⋅⎜────── - ──────⎟⎥
⎢    ⎝2⋅m_R1   2⋅m_L2⎠⎥
⎢                     ⎥
⎢    ⎛  1        1   ⎞⎥
⎢Π₂₃⋅⎜────── - ──────⎟⎥
⎢    ⎝2⋅m_R2   2⋅m_L3⎠⎥
⎢                     ⎥
⎢    ⎛  1        1   ⎞⎥
⎢Π₃₄⋅⎜────── - ──────⎟⎥
⎣    ⎝2⋅m_R3   2⋅m_L4⎠⎦

In [11]:
QZ12 = QZ[(Nx-N_lambda)::, 0:(Nx-N_lambda)]
RHS = QZ12 * Z1
display(RHS)

⎡    ⎛  1        1   ⎞⎤
⎢Π₁₂⋅⎜────── - ──────⎟⎥
⎢    ⎝2⋅m_R1   2⋅m_L2⎠⎥
⎢                     ⎥
⎢    ⎛  1        1   ⎞⎥
⎢Π₂₃⋅⎜────── - ──────⎟⎥
⎢    ⎝2⋅m_R2   2⋅m_L3⎠⎥
⎢                     ⎥
⎢    ⎛  1        1   ⎞⎥
⎢Π₃₄⋅⎜────── - ──────⎟⎥
⎣    ⎝2⋅m_R3   2⋅m_L4⎠⎦

In [12]:
o_label = []
for ind in range(1, N_lambda+1):
    str_symb = 'RHS_' + str(ind)        
    symb = sy.symbols(str_symb, **PPTY_STATE_VAR)
    o_label.append(symb)
o_label

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

[RHS₁, RHS₂, RHS₃]

In [13]:
QZ22 = QZ[Nx-N_lambda::, Nx-N_lambda::]
QZ22

⎡ 1      1                            ⎤
⎢──── + ────       0            0     ⎥
⎢m_R1   m_L2                          ⎥
⎢                                     ⎥
⎢              1      1               ⎥
⎢     0       ──── + ────       0     ⎥
⎢             m_R2   m_L3             ⎥
⎢                                     ⎥
⎢                           1      1  ⎥
⎢     0            0       ──── + ────⎥
⎣                          m_R3   m_L4⎦

## Building Core

In [14]:
label = 'FPUT'
core = phs.Core(label=label)

Storage components

In [15]:
core.add_storages(list(Z1), HZ)
core.set_Jxx(Jxx)

Observers

In [16]:
observers_dict = {}
for i in range(N_lambda):
    observers_dict[o_label[i]] = RHS[i]
    
for i in range(N_lambda):
    observers_dict[Z2[i]] = o_label[i]
    
core.add_observer(observers_dict)

display(core.observers)

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

OrderedDict(⎡⎛          ⎛  1        1   ⎞⎞  ⎛          ⎛  1        1   ⎞⎞  ⎛          ⎛  1
⎢⎜RHS₁, Π₁₂⋅⎜────── - ──────⎟⎟, ⎜RHS₂, Π₂₃⋅⎜────── - ──────⎟⎟, ⎜RHS₃, Π₃₄⋅⎜───
⎣⎝          ⎝2⋅m_R1   2⋅m_L2⎠⎠  ⎝          ⎝2⋅m_R2   2⋅m_L3⎠⎠  ⎝          ⎝2⋅m

        1   ⎞⎞                                       ⎤
─── - ──────⎟⎟, (Δ₁₂, RHS₁), (Δ₂₃, RHS₂), (Δ₃₄, RHS₃)⎥
_R3   2⋅m_L4⎠⎠                                       ⎦)

Subs

In [17]:
subs = {}
for symb in params:
    subs[symb] = 1

for k in k_vec:
    subs[k] = 100

display(subs)

core.subs.update(subs)

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

{k₁: 100, k₂: 100, k₃: 100, k₄: 100, m_L1: 1, m_L2: 1, m_L3: 1, m_L4: 1, m_R1:
 1, m_R2: 1, m_R3: 1, m_R4: 1}

In [18]:
core.x

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

[Π_L1, Π₁₂, Π₂₃, Π₃₄, Π_R4, xₖ₁, xₖ₂, xₖ₃, xₖ₄]

In [31]:
inits = {'x' : [0.0 for i in range(len(core.x))]}
inits['Gnl'] = [0.0 for i in range(len(core.x))]
inits['dx'] = [0.0 for i in range(len(core.x))]
inits['vnl'] = [0.0 for i in range(len(core.x))]
inits['dxH'] = [0.0 for i in range(len(core.x))]

ind = 2*Ns - N_lambda + 1
ind = 0
display(core.x[ind])
#inits['x'][ind] = 0.1

TypeError: expected str, bytes or os.PathLike object, not _io.BytesIO

Π_L1

In [32]:
config = {'fs'   : 1e5, # sample rate (Hz)
          'path' : os.path.join(os.getcwd(), '03_data'), # folder for output results/build
          'lang' : 'c++',
          'cmake': r'cmake',
          'grad' : 'discret',
          'maxit': 10,
          'pbar' : False,
          'split': False,
          'timer': True,
          'epsdg': np.power(np.finfo(np.float64).eps, 0.5),
          'eps'  : np.finfo(np.float64).eps,
         }

simu = core.to_simulation(
                          config, 
                          erase = True,
                          inits = inits
                          )

Build method FPUT...
    Init Method...
    Build numerical structure...
    Init update actions...
    Init arguments...
        Build x
        Build dx
        Build w
        Build u
        Build p
        Build vnl
        Build o
    Init functions...
        Build dxH
        Build ud_o
        Build jacGnlnl
        Build z
        Build Gnl
        Build y
Prepare method FPUT for C++ generation...
    Init value for x
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Init value for dx
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Init value for ud_x
    Init value for o
[0, 0, 0, 0, 0, 0]
    Init value for ud_o
[0, 0, 0, 0, 0, 0]
    Init value for Gnl
[0, 0, 0, 0, 0, 0, 0, 0, 0]
    Init value for Fnl
    Init value for res_Fnl
    Init value for save_Fnl
    Init value for jacGnlnl
    Init value for jacFnl
    Init value for ijacFnl
    Init value for vnl
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Init value for ud_vnl
    Init value for step_Fnl
    Init value

In [ ]:
import modify_core_solver as modifs
from importlib import reload  
reload(modifs)
folder_path_in = os.path.join(config['path'],label.lower(), 'src')
modifs.insert_solver(folder_path_in, Ns)

In [ ]:
tmax = 1.
Nit = int(tmax*config['fs'])
t_vec = np.linspace(0, tmax, int(Nit))


print('Build de la simulation à {0}'.format(datetime.datetime.now()))
simu.init(
        nt    = Nit,
        )
print('Simulation démarée à {0}'.format(datetime.datetime.now()))
simu.process()
print('Simulation finie à {0}'.format(datetime.datetime.now()))

In [ ]:
# Date, heure etc...
timedate_now = datetime.datetime.now()
month  = timedate_now.month
day    = timedate_now.day
hour   = timedate_now.hour
minute = timedate_now.minute
date_now = [str(month).zfill(2), str(day).zfill(2), str(hour).zfill(2), str(minute).zfill(2)]

filename = '03_data/simu.{0}_{1}.{2}_{3}.h5'.format(*date_now)
fid      = h5py.File(filename, 'x')
dt       = h5py.string_dtype()

# Creating groups
data_simu     = fid.create_group('data_simu')
metadata      = fid.create_group('metadata')
model         = fid.create_group('model')
fid.flush()

# Saving signals
data = np.array( list(simu.data.x())     , dtype=np.float64).squeeze()
data_simu.create_dataset("x", data=data)

data = np.array( list(simu.data.dx())    , dtype=np.float64).squeeze()
data_simu.create_dataset("dx", data=data)

data = np.array( list(simu.data.dxH())   , dtype=np.float64).squeeze()
data_simu.create_dataset("dxH", data=data)

data = np.array( list(simu.data.dtx())   , dtype=np.float64).squeeze()
data_simu.create_dataset("dtx", data=data)


# time vector
data = t_vec
data_simu.create_dataset("t", data=data)

data

data = inits['x']
data_simu.create_dataset("init_X", data=data)


subs = subs
place = model.create_group("subs")

for key, val in subs.items():
    place.create_dataset(name=str(key), data=val)
    
# labels
str_x_label = []
str_o_label = []


Nx = len(core.x)
model.create_dataset("Nx", (1,), data=Nx)


for label in core.x:
    str_x_label.append(str(label).encode('ascii'))
for label in core.o():
    str_o_label.append(str(label))

data = np.array(str_x_label, dtype='S')
model.create_dataset("x_label", shape=data.shape, data=data, dtype=dt)

data = np.array(str_o_label, dtype='S')
model.create_dataset("o_label", shape=data.shape, data=data, dtype=dt)

### Config de la simu
subs = config
metadata.create_dataset("config",(1,),data=str(subs), dtype=dt)

data = phs.__version__
metadata.create_dataset("pyphs_version",(1,),data=str(data), dtype=dt)

data = Nit
metadata.create_dataset("Nit", data=data)



data = date_now
for i, el in enumerate(data):
    data[i] = int(el)
metadata.create_dataset("date_now", data=data)

del data
fid.flush()
fid.close()
%store filename